In [3]:
# This example considers the following nonconvex nonlinear problem
#
#   maximize   2 x + y
#   subject to exp(x) + 4 sqrt (y) <= 9
#              x, y >= 0
#
#   We show you two approaches to solve this :
#
#   1) Use a piecewise - linear approach to handle general function
#      constraints ( such as exp and sqrt ).
#      a) Add two variables
#         u = exp(x)
#         v = sqrt (y)
#      b) Compute points (x, u) of u = exp(x) for some step length (e.g., x
#         = 0, 1e -3, 2e -3, ... , xmax ) and points (y, v) of v = sqrt (y) for
#         some step length (e.g., y = 0, 1e -3, 2e -3, ... , ymax ). We need to
#         compute xmax and ymax ( which is easy for this example , but this
#         does not hold in general ).
#      c) Use the points to add two general constraints of type
#         piecewise - linear .
#
#   2) Use the Gurobis built-in general function constraints directly ( EXP
#      and POW ). Here , we do not need to compute the points and the maximal
#      possible values , which will be done internally by Gurobi . In this
#      approach , we show how to "zoom in" on the optimal solution and
#      tighten tolerances to improve the solution quality .

In [4]:
import math
import gurobipy as gp
from gurobipy import GRB

In [17]:
def printsol(m, x, y, u, v):
    print('x = ' + str(x.x) + ', u = ' + str(u.x))
    print('y = ' + str(y.x) + ', v = ' + str(v.x))
    print('Obj = ' + str(m.objVal))
    
    # Calculate violstion of exp(x) + 4 sqrt(y) <= 9
    vio =math.exp(x.x) + 4 * math.sqrt(y.x) - 9
    if vio < 0:
        vio = 0
    print('Vio = ' + str(vio))

In [6]:
m = gp.Model('gc_pwl')

Using license file /Users/yj/gurobi.lic


In [7]:
x = m.addVar(name='x')
y = m.addVar(name='y')
u = m.addVar(name='u')
v = m.addVar(name='v')

In [8]:
m.setObjective(2 * x + y, GRB.MAXIMIZE)

In [10]:
lc = m.addConstr(u + 4 * v <= 9)

In [13]:
# Approach 1) PWL constraint approach

xpts = []
ypts = []
upts = []
vpts = []

intv = 1e-3

xmax = math.log(9)
t = 0.0
while t < xmax + intv:
    xpts.append(t)
    upts.append(math.exp(t)) # 核心步骤：这里串联起了变量与变量之间的关系
    t += intv

ymax = (9.0/4)*(9.0/4)
t = 0.0
while t < ymax + intv:
    ypts.append(t)
    vpts.append(math.sqrt(t)) # 核心步骤：这里串联起了变量与变量之间的关系
    t += intv

In [14]:
gc1 = m.addGenConstrPWL(x, u, xpts, upts, 'gc1')
gc2 = m.addGenConstrPWL(y, v, ypts, vpts, 'gc2')

In [15]:
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x36a07eee
Model has 2 general constraints
Variable types: 4 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+00, 9e+00]
Presolve added 3 rows and 7261 columns
Presolve time: 0.03s
Presolved: 5 rows, 7265 columns, 14526 nonzeros
Presolved model has 2 SOS constraint(s)
Variable types: 7265 continuous, 0 integer (0 binary)

Root relaxation: objective 9.275728e+00, 50 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.59665    0    1          -    5.59665      -     -    0s
     0     0    5.59665    0    1          -    5.59665      -     -    0s
     0     2    5.59665    0    1          

In [18]:
printsol(m, x, y, u, v)

x = 0.8842823139793613, u = 2.421246318640817
y = 2.704999999999813, v = 1.6446884203397958
Obj = 4.4735646279585355
Vio = 0


In [19]:
# Approach 2) General function constraint approach with auto PWL
#             translation by Gurobi

# restore unsolved state and get rid of PWL constraints
m.reset()
m.remove(gc1)
m.remove(gc2)
m.update()

Discarded solution information


In [21]:
# u = exp(x)
gcf1 = m.addGenConstrExp(x, u, name='gcf1')
# v = sqrt(y)
gcf2 = m.addGenConstrPow(y, v, 0.5, name='gcf2')

# use the equal piece length approach with the length = 1e-3


# FuncPieceLength attribute.
m.params.FuncPieces = 1 # Uses a fixed width for each piece; 
                        # the actual width is provided in the
                        # FuncPieceLength attribute.
m.params.FuncPieceLength = 1e-3

Changed value of parameter FuncPieces to 1
   Prev: 0  Min: -2  Max: 200000000  Default: 0
Parameter FuncPieceLength unchanged
   Value: 0.001  Min: 1e-05  Max: 1000000.0  Default: 0.01


In [22]:
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xfed95b74
Model has 4 general constraints
Variable types: 4 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+00, 9e+00]
Presolve added 11 rows and 14526 columns
Presolve time: 0.04s
Presolved: 13 rows, 14530 columns, 43580 nonzeros
Presolved model has 4 SOS constraint(s)
Variable types: 14530 continuous, 0 integer (0 binary)

Root relaxation: objective 5.599522e+00, 21 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.59665    0    2          -    5.59665      -     -    0s
     0     0    5.59665    0    2          -    5.59665      -     -    0s
     0     2    5.59665    0    2     

In [23]:
printsol(m, x, y, u, v)

x = 0.8927974379795071, u = 2.4419515097858566
y = 2.688, v = 1.6395121225535358
Obj = 4.473594875959014
Vio = 0


In [24]:
# Zoom in , use optimal solution to reduce the ranges and use a smaller
# pclen =1 -5 to solve it

x.lb = max(x.lb, x.x-0.01)
x.ub = min(x.ub, x.x+0.01)
y.lb = max(y.lb, y.x-0.01)
y.ub = min(y.ub, y.x+0.01)
m.update()
m.reset()

Discarded solution information


In [25]:
m.params.FuncPieceLength = 1e-5

Changed value of parameter FuncPieceLength to 1e-05
   Prev: 0.001  Min: 1e-05  Max: 1000000.0  Default: 0.01


In [26]:
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xb9a25613
Model has 4 general constraints
Variable types: 4 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [9e-01, 3e+00]
  RHS range        [9e+00, 9e+00]
Presolve added 11 rows and 8004 columns
Presolve time: 0.08s
Presolved: 13 rows, 8008 columns, 24014 nonzeros
Presolved model has 4 SOS constraint(s)
Variable types: 8008 continuous, 0 integer (0 binary)

Root relaxation: objective 4.473604e+00, 30 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.47360    0    4          -    4.47360      -     -    0s
     0     0    4.47360    0    4          -    4.47360      -     -    0s
     0     0    4.47360    0    1        

In [27]:
printsol(m, x, y, u, v)

x = 0.8912274379795071, u = 2.4381204570476833
y = 2.6910100000000003, v = 1.6404298217235629
Obj = 4.4734648759590145
Vio = 0
